# Maze challenge report - Diego Arcelli

Per risolvere la challange, dopo una prima fase di studio con piccoli script atti a comprendere i vari attributi delle celle del labirinto, piuttosto che implementare una per una le soluzioni di ciascuna quest ho preferito realizzare un unico programma interattivo (prendendo spunto dall'ultima quest) che ingloba le varie soluzioni delle quest.
Ho qundi realizzato semplice programma che permette di esplorare il labirinto, stampandone una sua rappresentazione grafica nel terminale man mano che si esplora, inserendo in input gli spostamenti desiderati.

L'esplorazione del labirinto è gestita principalmente dalla funzione exploration, che consiste essenzialmente di un ciclo che ad ogni iterazione stampa la parte di labirinto attualmente esplorata, richiede all'utente di inserire la prossima direzione nella quale spostarsi e permette di visualizzare le statistiche sul labirinto raccolte durante l'esplorazione.
Un altro elemento fondamentale è la lista explored_maze, che contiene tutte le celle del labirinto esplorate. Ogni elemento della lista è un dizionario che contiene le coordinate della cella ed il suo colore.

In [ ]:
def exploration(command):

    explored_maze = []
    exit = False
    getch = G._Getch()

    while not exit:

        res = mazeClient.send_command(command.GET_STATE)
        pos = json.loads(res)
        print(pos)

        elm = {}
        elm["x"] = pos["userX"]
        elm["y"] = pos["userY"]
        elm["val"] = pos["userVal"]
        if is_in_maze(explored_maze, elm) == False:
            explored_maze.append(elm)
        
        for block in pos["Neighbors"]:
            if is_in_maze(explored_maze, block) == False:
                explored_maze.append(block)
        
        os.system("clear")
        print("")
        print_map(explored_maze, elm)
        print("")
        values = get_stats(explored_maze)
        print("Total: " + str(values["total"]) + ", red: " + str(values["red"]) + ", green: " + str(values["green"]) + ", blue: " + str(values["blue"]) + ", white: " + str(values["white"]))
        filter_neighbors(pos["Neighbors"],elm)
        print("Select option:\nW) Up\nS) Down\nR) Right\nA) Left\nE) Exit\nP) Plot stats\n")
        sel = getch()
        if sel == "w" or sel == "W":
            mazeClient.send_command(command.MOVE_UP)
        elif sel == "s" or sel == "S":
            mazeClient.send_command(command.MOVE_DOWN)
        elif sel == "d" or sel == "D":
            mazeClient.send_command(command.MOVE_RIGHT)
        elif sel == "a" or sel == "A":
            mazeClient.send_command(command.MOVE_LEFT)
        elif sel == "e" or sel == "E":
            exit = True
        elif sel == "p" or sel == "P":
            frequency_distribution(explored_maze)
        else:
            print("Invalid")   

Ora verrano mostrate più in dettaglio le funzioni utilizzate dalla funzione exploration.

La funzione find_dimension prende come paramtero la lista delle celle esplorate e la usa per calcolare la dimensione della più piccola matrice rettangolare che può contenere tutte le celle attualmente esplorate. 

In [ ]:
def find_dimension(explored_maze):
    x_max = explored_maze[0]["x"]
    y_max = explored_maze[0]["y"]
    x_min = explored_maze[0]["x"]
    y_min = explored_maze[0]["y"]
    for elm in explored_maze:
        if elm["x"] > x_max:
            x_max = elm["x"]
        if elm["x"] < x_min:
            x_min = elm["x"]
        if elm["y"] > y_max:
            y_max = elm["y"]
        if elm["y"] < y_min:
            y_min = elm["y"]
    return x_max, x_min, y_max, y_min

La funzione print_map prima inserisce le celle esplorate in una matrice e poi la usa per stampare una rappresentazione grafica del labirinto esplorato, rappresentando la posizione del giocatore nel labirinto con il carattere '0'.

In [ ]:
def print_map(explored_maze, pos):

    x = pos["x"]
    y = pos["y"]

    x_max, x_min, y_max, y_min = find_dimension(explored_maze)
    lenght = x_max - x_min + 1
    width = y_max - y_min + 1

    size = (width, lenght)

    map = np.zeros(size)

    for i in reversed(range(lenght)):
        for j in reversed(range(width)):
            check = False
            for elm in explored_maze:
                if elm["x"] == i + x_min and elm["y"] == j + y_min:
                    check = True
            if check:
                map[j,i] = 1
            else:
                map[j,i] = 0

    for i in range(width + 2):
        print("@", end="")
    print("")
    for i in reversed(range(lenght)):
        print("@", end="")
        for j in reversed(range(width)):
            if i == x - x_min and j == y - y_min:
                print("0", end="")
            elif map[j,i] == 1:
                print(" ", end = "")
            else:
                print("#", end = "")         
        print("@")
    for i in range(width + 2):
        print("@", end="")


La funzione is_in_maze verifica se una determinata cella è già presente nella lista delle celle esplorate.

In [ ]:
def is_in_maze(explored_maze, block):
    for elm in explored_maze:
        if elm["x"] == block["x"] and elm["y"] == block["y"]:
            return True
    return False

La funzione filter_neighbors data una cella e la lista delle sue celle adiacenti, va ad eliminare dalla lista le celle adiacenti non raggiungibili. Questo perchè quando il challenge engine ritrona le informazioni id una cella, nella lista dei vicini inserisce anche le celle adiacenti sulla diagonale, che però non sono raggiungibili con un solo spostamento in quanto ci si può muovere solo in 4 direzioni. 

In [ ]:
def filter_neighbors(neighbors, block):
    filtered = []
    for elm in neighbors:
        if not(elm["x"] != block["x"] and elm["y"] != block["y"]):
            filtered.append(elm)
    available_directions(filtered, block)
    return filtered

La funzione aviable_direction stampa a schermo la direzioni delle celle raggiungibili a partire da una data cella con un solo spostamento.

In [ ]:
def available_directions(neighbors, block):
    print("Aviable direction: ", end="")
    for elm in neighbors:
        if elm["x"] > block["x"]:
            print("up ", end="")
        elif elm["x"] < block["x"]:
            print("down ", end="")
        elif elm["y"] > block["y"]:
            print("left ", end="")
        elif elm["y"] < block["y"]:
            print("right ", end="")
    print("")

La funzione get_stats calcola in numero totale di celle ed il numero di celle per colore presenti nella lista delle celle esplorate.

In [ ]:
def get_stats(explored_maze):
    values = {}
    values["total"] = 0
    values["red"] = 0
    values["white"] = 0
    values["blue"] = 0
    values["green"] = 0
    for block in explored_maze:
        values["total"] += 1
        if block["val"] == 82:
            values["red"] += 1
        elif block["val"] == 32:
            values["white"] += 1
        elif block["val"] == 71:
            values["green"] += 1
        elif block["val"] == 66:
            values["blue"] += 1
    return values

La funzione frequency_distribution semplicemente plotta un istogramma del numero delle celle esplorate per colore.

In [ ]:
def frequency_distribution(explored_maze):
    values = get_stats(explored_maze)
    absFreq = ["red"]*values["red"] + ["green"]*values["green"] + ["white"]*values["white"] + ["blue"]*values["blue"] 
    plt.hist(absFreq, density=False)
    plt.show()

Tra i file è anche presente Getch.py che contiente la seguente classe.

In [ ]:
class _Getch:

    def __init__(self):
        self.impl = _GetchUnix()

    def __call__(self): return self.impl()


class _GetchUnix:
    def __init__(self):
        import tty, sys

    def __call__(self):
        import sys, tty, termios
        fd = sys.stdin.fileno()
        old_settings = termios.tcgetattr(fd)
        try:
            tty.setraw(sys.stdin.fileno())
            ch = sys.stdin.read(1)
        finally:
            termios.tcsetattr(fd, termios.TCSADRAIN, old_settings)
        return ch

Essenzialmente mi serve per poter prendere l'input dell'utente da tastiera senza dover ogni volta premere il tasto invio, velocizzando l'esplorazione del labirinto. Però il modulo che utilizza per svolgere questa operazione è compatibilie solo con Linux (personalmente l'ho testato solo sulla mia macchina con Debian 10), dunque ho anche inserito il file main2.py che essenzialmente è identico a main.py solo che per l'input da tastiera usa la funzione input() di python e quindi ogni volta va confermato l'input premendo invio ma almeno è compatibile anche con Windows e Mac.